<a href="https://colab.research.google.com/github/Jaisrinivaspk/Dataspark_project/blob/main/Dataspark_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataspark EDA project


Importing the Libraries


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

1)Load All Datasets


In [2]:
customers = pd.read_csv('Customers.csv', encoding='latin1')
products = pd.read_csv('Products.csv', encoding='latin1')
sales = pd.read_csv('Sales.csv', encoding='latin1')
stores = pd.read_csv('Stores.csv', encoding='latin1')
exchange_rates = pd.read_csv('Exchange_Rates.csv', encoding='latin1')

2)Data Cleaning and Preparation

In [4]:
#CUSTOMERS
customers = customers.dropna(subset=['State Code'])
customers['Birthday'] = pd.to_datetime(customers['Birthday'], errors='coerce')
for col in ['Gender','Name','City','State Code','State','Zip Code','Country','Continent']:
    customers[col] = customers[col].astype(str).str.strip()

#PRODUCTS
products['Unit Cost USD'] = products['Unit Cost USD'].replace(r'[\$,]', '', regex=True).astype(float)
products['Unit Price USD'] = products['Unit Price USD'].replace(r'[\$,]', '', regex=True).astype(float)

#SALES
sales['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')
sales['Delivery Date'] = pd.to_datetime(sales['Delivery Date'], errors='coerce')

#STORES
stores['Square Meters'] = stores['Square Meters'].fillna(stores['Square Meters'].mean())
stores['Open Date'] = pd.to_datetime(stores['Open Date'], errors='coerce')

#EXCHANGE RATES
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'], errors='coerce')

3)Save Cleaned Data

In [5]:
customers.to_csv("clean_customers.csv", index=False)
products.to_csv("clean_products.csv", index=False)
sales.to_csv("clean_sales.csv", index=False)
stores.to_csv("clean_stores.csv", index=False)
exchange_rates.to_csv("clean_exchange_rates.csv", index=False)

4)Exploratory Data Analysis

In [ ]:
#4.1)CUSTOMER DEMOGRAPHICS

# Gender distribution
sns.countplot(data=customers, x='Gender')
plt.title('Customer Gender Distribution')
plt.show()
# Age distribution
customers['Age'] = (pd.Timestamp('today') - customers['Birthday']).dt.days // 365
sns.histplot(customers['Age'].dropna(), bins=20)
plt.title('Customer Age Distribution')
plt.show()

#4.2)SALES TRENDS OVER TIME

# Orders per year
sales['Order Year'] = sales['Order Date'].dt.year
sales['Order Year'].value_counts().sort_index().plot(kind='bar')
plt.title('Orders per Year')
plt.xlabel('Year')
plt.ylabel('Number of Orders')
plt.show()

#4.3)ORDERS BY CURRENCY
sales['Currency Code'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Orders by Currency')
plt.xlabel('Currency')
plt.ylabel('Number of Orders')
plt.show()

#4.4)STORE SIZE DISTRIBUTION
sns.histplot(stores['Square Meters'], bins=20, kde=True)
plt.title('Distribution of Store Sizes')
plt.xlabel('Square Meters')
plt.ylabel('Number of Stores')
plt.show()

5)Data Integration for Revenue Analysis

In [8]:
#Merge Sales and Products
sales_products = pd.merge(
    sales,
    products[['ProductKey', 'Unit Price USD', 'Unit Cost USD', 'Product Name', 'Category', 'Subcategory']],
    on='ProductKey',
    how='left'
)

#Prepare Dates for Exchange Rate Merge
sales_products['Order Date Str'] = sales_products['Order Date'].dt.strftime('%-m/%-d/%Y')
exchange_rates['Date Str'] = exchange_rates['Date'].dt.strftime('%-m/%-d/%Y')

#Merge with Exchange Rates
sales_products = pd.merge(
    sales_products,
    exchange_rates.rename(columns={'Currency': 'Currency Code'}),
    left_on=['Order Date Str', 'Currency Code'],
    right_on=['Date Str', 'Currency Code'],
    how='left'
)

#Calculate Revenue, Cost, Profit in USD
sales_products['Revenue USD'] = sales_products['Unit Price USD'] * sales_products['Quantity'] / sales_products['Exchange']
sales_products['Cost USD'] = sales_products['Unit Cost USD'] * sales_products['Quantity'] / sales_products['Exchange']
sales_products['Profit USD'] = sales_products['Revenue USD'] - sales_products['Cost USD']

# Save Integrated Data
sales_products.to_csv('clean_sales_products.csv', index=False)

6)Load Data into SQL Database

In [ ]:
engine = create_engine('sqlite:///global_electronics.db')
customers.to_sql('customers', engine, if_exists='replace', index=False)
products.to_sql('products', engine, if_exists='replace', index=False)
sales.to_sql('sales', engine, if_exists='replace', index=False)
stores.to_sql('stores', engine, if_exists='replace', index=False)
exchange_rates.to_sql('exchange_rates', engine, if_exists='replace', index=False)
sales_products.to_sql('sales_products', engine, if_exists='replace', index=False)


# SQL QUERIES


Importing

In [10]:
import pandas as pd
from sqlalchemy import create_engine

Connect to the Existing Database File

In [11]:
engine = create_engine('sqlite:///global_electronics.db')

SQL Queries

In [12]:
#Get total sales by year
query = """
SELECT strftime('%Y', [Order Date]) AS Year, SUM([Revenue USD]) AS Total_Revenue
FROM sales_products
GROUP BY Year
ORDER BY Year;
"""
result = pd.read_sql(query, engine)
print(result)

#Demographic Distribution: Gender & Country
query = """
SELECT Gender, Country, COUNT(*) AS Customer_Count
FROM customers
GROUP BY Gender, Country
ORDER BY Country, Gender;
"""
print(pd.read_sql(query, engine))

#Customer Age Distribution
query = """
SELECT
  (strftime('%Y', 'now') - strftime('%Y', Birthday)) AS Age,
  COUNT(*) AS Count
FROM customers
WHERE Birthday IS NOT NULL
GROUP BY Age
ORDER BY Age;
"""
print(pd.read_sql(query, engine))


   Year  Total_Revenue
0  2016   7.235760e+06
1  2017   7.712690e+06
2  2018   1.337781e+07
3  2019   1.863287e+07
4  2020   9.491416e+06
5  2021   1.080794e+06
    Gender         Country  Customer_Count
0   Female       Australia             721
1     Male       Australia             699
2   Female          Canada             755
3     Male          Canada             798
4   Female          France             351
5     Male          France             319
6   Female         Germany             734
7     Male         Germany             739
8   Female           Italy             315
9     Male           Italy             320
10  Female     Netherlands             345
11    Male     Netherlands             388
12  Female  United Kingdom             938
13    Male  United Kingdom            1006
14  Female   United States            3355
15    Male   United States            3473
    Age  Count
0    24    184
1    25    255
2    26    236
3    27    233
4    28    216
..  ...    ...
63 